##### ΗΥ553 Interactive Computer Graphics
# Assignment 1
##### Zacharias Pervolarakis
##### This is a short Jupyter notebook briefly explaining parts of the code
---


### Homes for everyone

GameObject Entity (name will probably change) is a wrapper for an Entity that contains four basic components:
- Basic Transform
- Mesh
- Shader
- Vertex Array

Both the cube and the pyramid are GameObjects.

In [ ]:
class GameObjectEntity(Entity):
    def __init__(self, name=None, type=None, id=None) -> None:
        super().__init__(name, type, id);

        # Gameobject basic properties
        self._color          = [1, 1, 1]; # this will be used as a uniform var
        # Create basic components of a primitive object
        self.trans          = BasicTransform(name="trans", trs=util.identity());
        self.mesh           = RenderMesh(name="mesh");
        self.shaderDec      = ShaderGLDecorator(Shader(vertex_source=Shader.VERT_PHONG_MVP, fragment_source=Shader.FRAG_PHONG));
        self.vArray         = VertexArray();
        # Add components to entity
        scene = Scene();
        scene.world.createEntity(self);
        scene.world.addComponent(self, self.trans);
        scene.world.addComponent(self, self.mesh);
        scene.world.addComponent(self, self.shaderDec);
        scene.world.addComponent(self, self.vArray);

    @property
    def color(self):
        return self._color;
    @color.setter
    def color(self, colorArray):
        self._color = colorArray;

    def drawSelfGui(self, imgui):
        changed, value = imgui.color_edit3("Color", self.color[0], self.color[1], self.color[2]);
        self.color = [value[0], value[1], value[2]];

    def SetVertexAttributes(self, vertex, color, index, normals = None):
        self.mesh.vertex_attributes.append(vertex);
        self.mesh.vertex_attributes.append(color);
        if normals is not None:
            self.mesh.vertex_attributes.append(normals);
        self.mesh.vertex_index.append(index);

To easily spawn gameobjects at will, I created a Singleton class called PrimitiveGameObjectSpawner. It uses the PrimitiveGameObjectType (Enum) to define what it spawns.

Then, a simple SpawnHome function was created that simply spawns a cube and a pyramid (offseted up). Finally, it returns the newly created Entity Home that contains the GameObjects.

In [ ]:

def SpawnHome():
    scene = Scene();

    home = scene.world.createEntity(Entity("Home"));
    scene.world.addEntityChild(scene.world.root, home);

    # Add trs to home
    trans = BasicTransform(name="trans", trs=util.identity());    scene.world.addComponent(home, trans);

    # Create simple cube
    cube: GameObjectEntity = PrimitiveGameObjectSpawner().Spawn(PrimitiveGameObjectType.CUBE);
    scene.world.addEntityChild(home, cube);

    # Create simple pyramid
    pyramid: GameObjectEntity = PrimitiveGameObjectSpawner().Spawn(PrimitiveGameObjectType.PYRAMID);
    scene.world.addEntityChild(home, pyramid);
    pyramid.trans.trs = util.translate(0, 1, 0); # Move pyramid to the top of the cube

    return home;


----


### Let there be light

To implement the shaders I heavily followed this tutorial:
https://learnopengl.com/Lighting/Basic-Lighting

The following classes were created:
Light(Entity) and PointLight(Light)

In [ ]:
class Light(Entity):
    def __init__(self, name=None, type=None, id=None) -> None:
        super().__init__(name, type, id);
        # Add variables for light
        self.color = [1, 1, 1];
        self.intensity = 1;
    
    def drawSelfGui(self, imgui):
        changed, value = imgui.slider_float("Intensity", self.intensity, 0, 10, "%.1f", 1);
        self.intensity = value;

        changed, value = imgui.color_edit3("Color", self.color[0], self.color[1], self.color[2]);
        self.color = [value[0], value[1], value[2]];
        None;
        
class PointLight(Light):
    def __init__(self, name=None, type=None, id=None) -> None:
        super().__init__(name, type, id);

        # Create basic components of a primitive object
        self.trans          = BasicTransform(name="trans", trs=util.identity());
        scene = Scene();
        scene.world.createEntity(self);
        scene.world.addComponent(self, self.trans);

    def drawSelfGui(self, imgui):
        super().drawSelfGui(imgui);


Their values can be customized from the GUI and are passed as uniform variables to the shaders that require them.

You can check the shaders for the code:
- VERT_PHONG_MVP
- FRAG_PHONG

### Important to note
Even though nobody has told us before, pyglGA uses indexed geometry to pass information to the GUI. That means that it is impossible to pass the normals of the vertices like we were expected to or like we used to do it in HY338. 

Indexed geometry has only one index for all the buffers, so if we have only 8 vertices we cant represent more than one normal for each.

For counteract this, I created a simple IndexConverter class that receives:
- array of vertices
- array of colors
- array of indices,
and it converts the above for information that can be fed into the indexed geometry glBuffers.

For now it simlpy does the job but it is not optimized. Normally it should find similar pairs and use the same index. Maybe I will fix it on the next assignment.

In [1]:
class IndexedConverter():
    
    # Assumes triangulated buffers. Produces indexed results that support
    # normals as well.
    def Convert(self, vertices, colors, indices, produceNormals=True):

        iVertices = [];
        iColors = [];
        iNormals = [];
        iIndices = [];
        for i in range(0, len(indices), 3):
            iVertices.append(vertices[indices[i]]);
            iVertices.append(vertices[indices[i + 1]]);
            iVertices.append(vertices[indices[i + 2]]);
            iColors.append(colors[indices[i]]);
            iColors.append(colors[indices[i + 1]]);
            iColors.append(colors[indices[i + 2]]);
            iNormals.append(util.calculateNormals(vertices[indices[i]], vertices[indices[i + 1]], vertices[indices[i + 2]]));
            iNormals.append(util.calculateNormals(vertices[indices[i]], vertices[indices[i + 1]], vertices[indices[i + 2]]));
            iNormals.append(util.calculateNormals(vertices[indices[i]], vertices[indices[i + 1]], vertices[indices[i + 2]]));

            iIndices.append(i);
            iIndices.append(i + 1);
            iIndices.append(i + 2);

        iVertices = np.array(
            iVertices,
            dtype=np.float32
        )
        iColors = np.array(
            iColors,
            dtype=np.float32
        )
        iNormals = np.array(
            iNormals,
            dtype=np.float32
        )
        iIndices = np.array(
            iIndices,
            dtype=np.uint32
        );

        return iVertices, iColors, iNormals, iIndices;

---

### Transforming Pyramid

For this pyramid a primitive pyramid was spawned and a RotateAnimation class was attached to it. For it to match the ECSS system an Animation System needs to be created but who has time for that.

Every while loop, animation->Progress() is called.

In [ ]:
class RotateAnimation(Entity):
    def __init__(self, name=None, type=None, id=None) -> None:
        super().__init__(name, type, id);
        self._angle = 1;
        self._target = None;

    def SetTarget(self, target: BasicTransform):
        self._target = target;

    def Progress(self):
        self._target.trs = self._target.trs @ util.rotate((0, 1, 0), self._angle);

    def drawSelfGui(self, imgui):
        changed, value = imgui.slider_float("Euler Angle", self._angle, -20, 20, "%.1f", 1);
        self._angle = value;
    

Naturaly, it's values can be changed when clicked. Not much to it here.

---


### Effective GUI

No reason or time to get into detail. It works ¯\_(ツ)_/¯

---

### Camera GUI

A fast and simple as the name suggests camera class was created:

In [ ]:
class SimpleCamera(Entity):
    def __init__(self, name=None, type=None, id=None) -> None:
        super().__init__(name, type, id)
        scene = Scene();
        rootEntity = scene.world.root;

        scene.world.addEntityChild(rootEntity, self);

        entityCam1 = scene.world.createEntity(Entity(name="entityCam1"));
        scene.world.addEntityChild(self, entityCam1);
        self.trans1 = scene.world.addComponent(entityCam1, BasicTransform(name="trans1", trs=util.identity()));
        
        entityCam2 = scene.world.createEntity(Entity(name="entityCam2"));
        scene.world.addEntityChild(entityCam1, entityCam2);
        self.trans2 = scene.world.addComponent(entityCam2, BasicTransform(name="trans2", trs=util.identity()));
        
        self._near = 1;
        self._far = 20;

        self._fov = 50;
        self._aspect = 1.0;

        self._left = -10;
        self._right = 10;
        self._bottom = -10;
        self._top = 10;

        self._mode = "perspective";
        self._camera = scene.world.addComponent(entityCam2, Camera(util.perspective(self._fov, self._aspect, self._near, self._far), "MainCamera", "Camera", "500"));        
        None;

    @property
    def camera(self):
        return self._camera;

    def drawSelfGui(self, imgui):
        if imgui.button("Orthograpic") and self._mode is "perspective":
            self._mode = "orthographic";
            self._camera.projMat = util.ortho(self._left, self._right, self._bottom, self._top, self._near, self._far);
        if imgui.button("Perspective") and self._mode is "orthographic":
            self._mode = "perspective";
            self._camera.projMat = util.perspective(self._fov, self._aspect, self._near, self._far)

        if self._mode is "orthographic":
            changed, value = imgui.slider_float("Left", self._left, -50, -1, "%0.1f", 1);
            self._left = value;
            changed, value = imgui.slider_float("Right", self._right, 1, 50, "%0.1f", 1);
            self._right = value;
            changed, value = imgui.slider_float("Bottom", self._bottom, -50, -1, "%0.1f", 1);
            self._bottom = value;
            changed, value = imgui.slider_float("Top", self._top, 1, 50, "%0.1f", 1);
            self._top = value;

            self._camera.projMat = util.ortho(self._left, self._right, self._bottom, self._top, self._near, self._far);
        elif self._mode is "perspective":
            changed, value = imgui.slider_float("FOV", self._fov, 1, 120, "%0.1f", 1);
            self._fov = value;
            changed, value = imgui.slider_float("Aspect", self._aspect, 0.5, 2, "%0.1f", 1);
            self._aspect = value;

            self._camera.projMat = util.perspective(self._fov, self._aspect, self._near, self._far)


Code above is self explanatory^^